## Deliverable 2. Create a Customer Travel Destinations Map.

In [92]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [93]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather description,Country,Date
0,0,Barrow,71.2906,-156.7887,-5.78,77,40,16.11,haze,US,2022-04-04 02:19:56
1,1,Bintulu,3.1667,113.0333,84.61,79,20,3.44,few clouds,MY,2022-04-04 02:24:33
2,2,Geraldton,-28.7667,114.6000,73.81,83,40,18.41,scattered clouds,AU,2022-04-04 02:24:33
3,3,Rikitea,-23.1203,-134.9692,77.76,64,100,9.66,overcast clouds,PF,2022-04-04 02:24:28
4,4,Touros,-5.1989,-35.4608,76.84,91,87,8.75,overcast clouds,BR,2022-04-04 02:22:17


In [94]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [95]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Weather description,Country,Date
1,1,Bintulu,3.1667,113.0333,84.61,79,20,3.44,few clouds,MY,2022-04-04 02:24:33
3,3,Rikitea,-23.1203,-134.9692,77.76,64,100,9.66,overcast clouds,PF,2022-04-04 02:24:28
4,4,Touros,-5.1989,-35.4608,76.84,91,87,8.75,overcast clouds,BR,2022-04-04 02:22:17
5,5,Ternate,0.8000,127.4000,83.12,71,91,3.33,overcast clouds,ID,2022-04-04 02:24:33
6,6,Byron Bay,-28.6500,153.6167,78.71,66,52,5.41,broken clouds,AU,2022-04-04 02:24:34
...,...,...,...,...,...,...,...,...,...,...,...
665,665,Hua Hin,12.5707,99.9588,78.82,53,20,10.36,few clouds,TH,2022-04-04 02:41:51
666,666,Coxim,-18.5067,-54.7600,75.24,82,14,3.13,few clouds,BR,2022-04-04 02:41:51
668,668,Mulanur,10.7667,77.7167,82.60,65,93,1.21,overcast clouds,IN,2022-04-04 02:41:51
671,671,Paka,4.6374,103.4368,84.13,79,40,0.00,scattered clouds,MY,2022-04-04 02:41:52


In [96]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Weather description    0
Country                0
Date                   0
dtype: int64

In [97]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(axis=0)
clean_df.isnull().sum()

City_ID                0
City                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Weather description    0
Country                0
Date                   0
dtype: int64

In [98]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
clean_df=clean_df.rename(columns={'Weather description':'Current Description'})
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Bintulu,MY,84.61,few clouds,3.1667,113.0333,
3,Rikitea,PF,77.76,overcast clouds,-23.1203,-134.9692,
4,Touros,BR,76.84,overcast clouds,-5.1989,-35.4608,
5,Ternate,ID,83.12,overcast clouds,0.8000,127.4000,
6,Byron Bay,AU,78.71,broken clouds,-28.6500,153.6167,
9,Kavaratti,IN,82.51,scattered clouds,10.5669,72.6420,
11,Puerto Ayora,EC,76.55,overcast clouds,-0.7393,-90.3518,
19,Calabozo,VE,89.62,overcast clouds,8.9242,-67.4293,
23,Neiafu,TO,84.88,broken clouds,-18.6500,-173.9833,
25,Atuona,PF,79.88,few clouds,-9.8000,-139.0333,


In [99]:
# hotel_df.iloc[0]['city']

In [100]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}
index_list_no_hotel_name=[]
# 6b. Iterate through the hotel DataFrame.
for index,row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search.
    params["location"] = f"{lat},{lng}"
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try: 
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        print(f'index{index} - city:{hotel_df.loc[index]["City"]}  was successful')
    except:
#         print(hotel_df.loc[index]["City"])
        print(f'no nearby hotel found for the following coordination: lat ={lat}, long = {lng} -- index{index} - city:{hotel_df.loc[index]["City"]}')
        index_list_no_hotel_name.append(index)

index1 - city:Bintulu  was successful
index3 - city:Rikitea  was successful
index4 - city:Touros  was successful
index5 - city:Ternate  was successful
index6 - city:Byron Bay  was successful
index9 - city:Kavaratti  was successful
index11 - city:Puerto Ayora  was successful
index19 - city:Calabozo  was successful
index23 - city:Neiafu  was successful
index25 - city:Atuona  was successful
index28 - city:Vaini  was successful
index29 - city:West Bay  was successful
index41 - city:Kavieng  was successful
index43 - city:Oistins  was successful
index48 - city:Georgetown  was successful
index55 - city:Ladario  was successful
index57 - city:Hithadhoo  was successful
index60 - city:Maceio  was successful
index63 - city:Port Blair  was successful
index69 - city:Avarua  was successful
index70 - city:Hambantota  was successful
no nearby hotel found for the following coordination: lat =-20.2333, long = 164.0167 -- index71 - city:Poum
index80 - city:Bambous Virieux  was successful
index81 - city:Wa

index678 - city:Carutapera  was successful


In [105]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df=hotel_df.drop(index_list_no_hotel_name)
clean_hotel_df.head(5)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Bintulu,MY,84.61,few clouds,3.1667,113.0333,900 Boutique INN
3,Rikitea,PF,77.76,overcast clouds,-23.1203,-134.9692,People ThankYou
4,Touros,BR,76.84,overcast clouds,-5.1989,-35.4608,INN NEW HORIZON
5,Ternate,ID,83.12,overcast clouds,0.8000,127.4000,Sahid Bela Ternate Hotel
6,Byron Bay,AU,78.71,broken clouds,-28.6500,153.6167,Backpackers Inn


In [102]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [106]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City Name</dt><dd>{City}</dd>
<dt>country code</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [110]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))